## Тест: Практика построения регрессии
### Вопрос 1.
Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

О каждой из них мы знаем:

  -  сколько детей она родила (признак `ceb`)
  -  возраст (`age`)
  -  длительность получения образования (`educ`)
  -  религиозная принадлежность (`religion`)
  -  идеальное, по её мнению, количество детей в семье (`idlnchld`)
  -  была ли она когда-нибудь замужем (`evermarr`)
  -  возраст первого замужества (`agefm`)
  -  длительность получения образования мужем (`heduc`)
  -  знает ли она о методах контрацепции (`knowmeth`)
  -  использует ли она методы контрацепции (`usemeth`)
  -  живёт ли она в городе (`urban`)
  -  есть ли у неё электричество, радио, телевизор и велосипед (`electric, radio, tv, bicycle`)

Давайте научимся оценивать количество детей `ceb` по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак `religion`?

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('botswana.tsv', sep='\t')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [3]:
data.religion.unique()

array(['catholic', 'protestant', 'spirit', 'other'], dtype=object)

### Вопрос 2.
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


In [5]:
data.dropna().shape

(1834, 15)

### Вопрос 3.
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке `agefm` пропуски стоят только там, где `evermarr=0`, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака `NaN` соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

   - создать новый бинарный признак
    $$x_2=1,x_1='не применимо',0,иначе;$$
   - заменить "не применимо" в $x_1$ на произвольную константу $c$, которая среди других значений $x_1$ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида
$$y=\beta_0+\beta_1 x_1+\beta_2x_2,$$
на тех объектах, где $x_1$ было измерено, регрессионное уравнение примет вид
$$y=\beta_0+\beta_1x,$$
а там, где $x_1$ было "не применимо", получится
$$y=\beta_0+\beta_1c+\beta_2.$$
Выбор c влияет только на значение и интерпретацию $\beta_2$, но не $\beta_1$.

Давайте используем этот метод для обработки пропусков в `agefm` и `heduc`.

 1.   Создайте признак `nevermarr`, равный единице там, где в `agefm` пропуски.
 2.   Удалите признак `evermarr` — в сумме с `nevermarr` он даёт константу, значит, в нашей матрице `X` будет мультиколлинеарность.
 3.   Замените `NaN` в признаке `agefm` на $c_{agefm}=0$.
 4.   У объектов, где `nevermarr = 1`, замените `NaN` в признаке `heduc` на $c_{heduc_{1}}=−1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке `heduc`?

In [6]:
data['nevermarr'] = map(lambda x: 1 if np.isnan(x) else 0, data['agefm'])
data.drop(['evermarr'], axis=1, inplace=True)

In [7]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [8]:
data['agefm'].fillna(0.0, inplace=True)
for i, item in enumerate(data['nevermarr']):
    if item == 1:
        data.heduc[i] = -1.0
data.head()

C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb          4361 non-null int64
age          4361 non-null int64
educ         4361 non-null int64
religion     4361 non-null object
idlnchld     4241 non-null float64
knowmeth     4354 non-null float64
usemeth      4290 non-null float64
agefm        4361 non-null float64
heduc        4238 non-null float64
urban        4361 non-null int64
electric     4358 non-null float64
radio        4359 non-null float64
tv           4359 non-null float64
bicycle      4358 non-null float64
nevermarr    4361 non-null int64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


In [10]:
4361-4238

123

### Вопрос 4.
Избавимся от оставшихся пропусков.

Для признаков `idlnchld, heduc` и `usemeth` проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (`idlnchld_noans, heduc_noans, usemeth_noans`), замените пропуски на нехарактерные значения ($c_{idlnchld}=−1$, $c_{heduc_{2}}=−2$ (значение -1 мы уже использовали), $c_{usemeth}=−1$).

Остались только пропуски в признаках `knowmeth, electric, radio, tv` и `bicycle`. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик `ceb`).

In [11]:
data['idlnchld_noans'] = map(lambda x: 1 if np.isnan(x) else 0, data['idlnchld'])
data['heduc_noans'] = map(lambda x: 1 if np.isnan(x) else 0, data['heduc'])
data['usemeth_noans'] = map(lambda x: 1 if np.isnan(x) else 0, data['usemeth'])
data.head(20)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0
5,1,30,5,spirit,5.0,1.0,0.0,24.0,7.0,1,1.0,0.0,0.0,0.0,0,0,0,0
6,3,42,4,other,3.0,1.0,0.0,15.0,11.0,1,1.0,0.0,1.0,0.0,0,0,0,0
7,1,36,7,other,4.0,1.0,1.0,24.0,9.0,1,1.0,0.0,0.0,0.0,0,0,0,0
8,4,37,16,catholic,4.0,1.0,1.0,26.0,17.0,1,1.0,1.0,1.0,1.0,0,0,0,0
9,1,34,5,protestant,4.0,1.0,1.0,18.0,3.0,1,0.0,1.0,0.0,0.0,0,0,0,0


In [12]:
for i, item in enumerate(data['idlnchld_noans']):
    if item == 1:
        data.idlnchld[i] = -1.0
for i, item in enumerate(data['heduc_noans']):
    if item == 1:
        data.heduc[i] = -2.0
for i, item in enumerate(data['usemeth_noans']):
    if item == 1:
        data.usemeth[i] = -1.0

C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
data.dropna(inplace=True)
data.shape

(4348, 18)

In [14]:
4348*18

78264

### Вопрос 5.
Постройте регрессию количества детей `ceb` на все имеющиеся признаки методом `smf.ols`, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

Если код из примера у вас не воспроизводится:

  -  убедитесь, что вы сделали так:
  ```
  import statsmodels.formula.api as smf
  ```

  -  возможно, вам нужно обновить библиотеку `patsy`; выполните в командной строке 
    ```
    pip install -U patsy
    ```


In [15]:
import statsmodels.formula.api as smf

m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc +'\
             'urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans +'\
             'heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 19 Oct 2017   Prob (F-statistic):               0.00
Time:                        16:03:11   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

### Вопрос 6.
Обратите внимание, что для признака `religion` в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [16]:
3

3

### Вопрос 7.
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

- Ошибка гомоскедастична, $p>0.05$

- **Ошибка гетероскедастична, $p≤0.05$, нужно делать поправку Уайта**

In [17]:
import statsmodels.stats.api as sms
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


### Вопрос 8.
Удалите из модели незначимые признаки `religion, radio` и `tv`. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [18]:
data.drop(['religion', 'radio', 'tv'], axis=1, inplace=True)

In [19]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc +'\
             'urban + electric + bicycle + nevermarr + idlnchld_noans +'\
             'heduc_noans + usemeth_noans', 
             data=data)
fitted2 = m2.fit(cov_type='HC1')
print fitted2.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 19 Oct 2017   Prob (F-statistic):               0.00
Time:                        16:03:12   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [20]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted2.resid, fitted2.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [21]:
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

F=0.919236, p=0.467231, k1=5.000000


### Вопрос 9.
Признак `usemeth_noans` значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в `usemeth` произвольно выбранным значением `cusemeth=−1`, удалять `usemeth_noans` и `usemeth` можно только вместе.

Удалите из текущей модели `usemeth_noans` и `usemeth`. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без `usemeth` и `usemeth_noans`. 

In [22]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc +'\
             'urban + electric + bicycle + nevermarr + idlnchld_noans +'\
             'heduc_noans', 
             data=data)
fitted3 = m3.fit(cov_type='HC1')
print fitted3.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Thu, 19 Oct 2017   Prob (F-statistic):               0.00
Time:                        16:03:13   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [23]:
print m2.fit().compare_f_test(m3.fit())

(92.890582301098021, 3.1552009480371243e-40, 2.0)


### Вопрос 10.
Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

- У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])

- Итоговая модель объясняет 63% вариации отклика

- __У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])__

- __У женщин, не знающих, какое количество детей идеально, в среднем на
$$\beta_{idlnchld_noans} + c_{idlnchld}\beta_{idlnchld}\thickapprox0.58$$
детей больше__

- __С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])__

- У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])

In [24]:
0.6565 + (-1)*0.0770

0.5795